In [1]:
import pandas as pd
import numpy as np
import re, os, pickle
from os.path import join
from tqdm import tqdm
from datetime import datetime

# 옛날거 반영해야 하므로 전산에서 다운 받는다.
# 필요컬럼은 채무자키,계좌키,합의서체결일,채무구분
# 합의서체결일자 있는 것과 주채무만 남기고 나머지는 삭제
# 주민번호는 비식별처리되어있으며, 계좌별로 합의서체결일이 다를 수 있으므로 계좌키로 작업한다.

account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}

In [6]:
# 파일읽기 #####################################################################################################
wd = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\시효\신복실효건 반영하기\대성"
credit = pd.read_excel(join(wd, "신용회복새창_20231030_1420.xlsx"), dtype=str)[["채무자키","계좌키","합의서체결일","채무구분"]]
# account = pd.read_pickle(r"D:\3.자산\전산 dataset\대성\231027\계좌조회새창_20231027_2057.pkl")[['채무자키','계좌키','시효완성일','시효연장사유','시효중단여부']]
account = pd.read_excel(r"D:\3.자산\전산 dataset\대성\231027\계좌조회새창_20231027_2057.xlsx", dtype=account_dtype)[['채무자키','계좌키','시효완성일','시효연장사유','시효중단여부']]
################################################################################################################

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [27]:
# 합의서체결일자 없는 거 지우기
# credit.합의서체결일[0]
print(len(credit))
credit.drop(credit[credit.합의서체결일.isna()].index, axis=0, inplace=True)
credit.drop(credit[credit.채무구분=="보증채무"].index, axis=0, inplace=True)
print(len(credit))

1313
1313


In [28]:
# 신복에서 시효완성일 및 시효연장사유 작성
credit["시효완성일"] = (pd.to_numeric(credit.합의서체결일.str[:4]) + 5).astype(str) +credit.합의서체결일.str[4:]
credit["시효연장사유"] = "신용회복 합의서 체결:"+credit.합의서체결일

# 계좌와 병합 후 시효수정할 것만 
merged = credit.merge(account, on=['채무자키','계좌키'], how='left', suffixes=['_신복','_계좌'])
시효수정 = merged.query('시효완성일_신복 > 시효완성일_계좌')
print("계좌키 중복제거 전 : ", len(시효수정))

# 시효완성일 역순정렬 후 중복계좌 드랍하기
시효수정 = 시효수정.sort_values(['시효완성일_신복'], ascending=False)
시효수정 = 시효수정.drop_duplicates(['계좌키'], keep='first')
print("계좌키 중복제거 후 : ", len(시효수정))

계좌키 중복제거 전 :  74
계좌키 중복제거 후 :  74


In [29]:
시효수정.rename(columns={'시효완성일_신복':'최초시효완성일', '시효연장사유_신복':'최초시효완성일메모'}, inplace=True)
###########################################################################
시효수정.to_excel(join(wd, "231030 신용회복 합의서체결일반영 시효수정_비교파일.xlsx"), index=False)
###########################################################################

#### 전산에서 종결건 제외 후 엑셀일괄수정 > 계좌정보